In [1]:
from IPython.display import JSON

In [2]:
import pandas as pd
import numpy as np
from dateutil import parser

#Data Visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
sns.set(style='darkgrid', color_codes=True)


#Google API
from googleapiclient.discovery import build

In [3]:
# NLP Libraries
import wordcloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to C:\Users\MZ
[nltk_data]     Khan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\MZ
[nltk_data]     Khan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
api_key = 'Your API KEY'

In [5]:
channel_ids = ['UC_mYaQAE6-71rjSN6CeCA-g']         #["UChfo46ZNOV-vtehDc25A1Ug"] #  "UCtYLUTtgS3k1Fg4y5tAhLbw" can be more channel IDs

In [6]:
api_service_name = "youtube"
api_version = "v3"


# Get credentials and create an API client

youtube = build(
    api_service_name, api_version, developerKey=api_key)



In [7]:
 def get_channel(youtube, channel_ids):
    all_data = []
        
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_ids
    )
    response = request.execute()
    
    for item in response['items']:
        data = {'channelName':item['snippet']['title'], 'subscribers' : item['statistics']['subscriberCount'], 
                'views': item['statistics']['viewCount'], 'totalVideos' : item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        all_data.append(data)
        
    return(pd.DataFrame(all_data))
    

In [8]:
channel_stats = get_channel(youtube, channel_ids)

In [9]:
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,NeetCode,217000,15077407,329,UU_mYaQAE6-71rjSN6CeCA-g


In [10]:
playlist_Id=  'UU_mYaQAE6-71rjSN6CeCA-g'    # 'UUhfo46ZNOV-vtehDc25A1Ug'  #  "UUhfo46ZNOV-vtehDc25A1Ug"

def get_video_ids(youtube, playlist_Id):
    
    video_ids = []


    request = youtube.playlistItems().list(
         part="snippet,contentDetails",
         playlistId = playlist_Id,
         maxResults=50
        
     )
    response = request.execute()
    
    
    
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
        
        
    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
                  part="snippet,contentDetails",
                  playlistId = playlist_Id,
                  maxResults=50, 
                  pageToken = next_page_token
                )
        
        
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])
        next_page_token = response.get('nextPageToken')
        
    return video_ids

In [11]:
video_ids = get_video_ids(youtube, playlist_Id)

In [12]:
video_ids

['4G5t1HwHQD4',
 'TKC98Sumg2A',
 'VUVpTZVa7Ls',
 '5ueSge4n4bs',
 'DmWirHqCq1s',
 'RegQckCegDk',
 'D4T2N0yAr20',
 'PFRzAysTvpo',
 'rMj6KnxZu5M',
 'nalYyLZgvCY',
 'KE8MQuwE2yA',
 'qU32rTy_kOM',
 'uhYq27iSk9s',
 'q5ANAl8Z458',
 'w6LcypDgC4w',
 'bwgjvzMSlMQ',
 '4SUJt3dP2Jc',
 'T53EyLPj4UM',
 '7vVEJwVvAlI',
 'U4hFQCa1Cq0',
 'nJYFh4Dl-as',
 'Id_tqGdsZQI',
 'Re_v0j0CRsg',
 '08sn_w4LWEE',
 'aa2ijyWBBIc',
 'JrxRYBwG6EI',
 '_d0T_2Lk2qA',
 'YUF3_eBdzsk',
 'RXy5RzGF5wo',
 's4DPM8ct1pI',
 'd-B_gk_gJtQ',
 'XbaxWuHIWUs',
 'gkINMhbbIbU',
 '68a1Dc_qVq4',
 'Z6idIicFDOE',
 'j4KwhBziOpg',
 'FzTYfsmtOso',
 'ft0owvS5tQA',
 'evUFsOb_iLY',
 'g_S5WuasWUE',
 '5rHz_6s2Buw',
 'EYeR-_1NRlQ',
 'OKcrLfR-8mE',
 'VD9NACqBCw4',
 'iu0082c4HDE',
 '8u-H6O_XQKE',
 'WDx6Y4i4xJ8',
 'nhAsMabiVkM',
 'aBbsfn863oA',
 'cFabMOnJaq0',
 'Kk6mXAzqX3Y',
 'vGsyTE4s34w',
 'slYh0ZNEqSw',
 'W_akoecmCbM',
 'xW4QsTtaCa4',
 'Cg_wWPHJ2Sk',
 'o-YDQzHoaKM',
 'QhPdNS143Qg',
 'aYqYMIqZx5s',
 'G9xeB2-7PqY',
 '8i-f24YFWC4',
 'Pzg3bCDY87w',
 'D0X0BO

In [13]:
len(video_ids)

329

In [14]:
request = youtube.videos().list(
    part="snippet,contentDetails,statistics",
    id=video_ids[0:5]
)
response = request.execute()

JSON(response)

<IPython.core.display.JSON object>

In [15]:
def get_video_details(youtube, video_ids):

    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description','tags', 'publishedAt'],
                            'statistics':['viewCount', 'likeCount', 'favouritrCount', 'commentCount'],
                            'contentDetails': ['duration', 'defination', 'caption']
                            }

            video_info = {}
            video_info['video_id']= video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    #print(v[:10])

                    try:
                        video_info[v] = video[k][v]
                        #print(video_info)
                    except:
                            video_info[v] = None

            all_video_info.append(video_info)
        return pd.DataFrame(all_video_info)                        



In [16]:
video_df = get_video_details(youtube, video_ids)
video_df.head()

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouritrCount,commentCount,duration,defination,caption
0,4G5t1HwHQD4,NeetCode,How I Coded a SaaS,🚀 https://neetcode.io/ - 25% OFF LAUNCH SALE\n...,None,2022-09-04T15:00:02Z,1585,158,None,16,PT13M15S,None,false
1,TKC98Sumg2A,NeetCode,CHANGE YOUR LIFE - Algorithms and Data Structu...,🚀 https://neetcode.io/ - 25% OFF LAUNCH SALE\n...,None,2022-08-30T14:00:13Z,22688,1164,None,148,PT5M20S,None,false
2,VUVpTZVa7Ls,NeetCode,Count Vowels Permutation - Dynamic Programming...,🚀 https://neetcode.io/ - A better way to prepa...,None,2022-08-07T01:47:15Z,24472,842,None,112,PT11M46S,None,false
3,5ueSge4n4bs,NeetCode,Working at Google - Tech Stack I Use as a Soft...,🚀 https://neetcode.io/ - A better way to prepa...,None,2022-07-19T14:00:08Z,183891,5023,None,377,PT7M15S,None,false
4,DmWirHqCq1s,NeetCode,Best FREE site for coding interviews,🚀 https://neetcode.io/ - A better way to prepa...,None,2022-07-06T15:00:06Z,36040,2684,None,343,PT6M25S,None,false


In [17]:
video_df.to_csv(r'C:\Users\MZ Khan\Desktop\Courses\YouTube API\neetcode.csv')

In [18]:
def get_comments_in_videos(youtube, video_ids):
    
    all_comments = []
    
    for video_id in video_ids:
        try:   
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id
            )
            response = request.execute()
        
            comments_in_video = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] for comment in response['items'][0:10]]
            comments_in_video_info = {'video_id': video_id, 'comments': comments_in_video}

            all_comments.append(comments_in_video_info)
            
        except: 
            # When error occurs - most likely because comments are disabled on a video
            print('Could not get comments for video ' + video_id)
        
    return pd.DataFrame(all_comments)    
        

In [ ]:
comments = get_comments_in_videos(youtube, video_ids)


In [19]:
comments

,video_id,comments
0,TKC98Sumg2A,[🚀 https://neetcode.io/ - 25% OFF LAUNCH SALE\...
1,VUVpTZVa7Ls,[🚀 https://neetcode.io/ - The best FREE site t...
2,5ueSge4n4bs,[What tech stack do you guys use?\n\n🚀 https:/...
3,DmWirHqCq1s,"[Damn, over 100 people already signed up in th..."
4,RegQckCegDk,"[Thanks!, your videos are extremely helpful to..."
...,...,...
323,KLlXCFG5TnA,[🚀 https://neetcode.io/ - I created a FREE sit...
324,zx5Sw9130L0,[🚀 https://neetcode.io/ - I created a FREE sit...
325,P6RZZMu_maU,[🚀 https://neetcode.io/ - I created a FREE sit...
326,m17yOR5_PpI,[💡 GRAPH PLAYLIST: https://www.youtube.com/wat...


In [20]:
comments.to_csv(r'C:\Users\MZ Khan\Desktop\Courses\YouTube API\neetcode_comment.csv')

## Data Preprocessing 

In [21]:
comments.loc[0]['comments']

["🚀 https://neetcode.io/ - 25% OFF LAUNCH SALE\n\nJust to clarify: I'll start making a lot more free videos on the channel, including OOP & System Design soon! The NC150 is free and will remain free forever, I also won't be deleting ANY of the current videos. The current and future courses, however, will be available to Pro members.",
 "Although I'm still a student, I bought prime as soon as I saw it. Because you really helped me a lot on my way to learn programming. Thank you very much!\n Hope I can find an internship soon.",
 'Hi NC, when the system design course will be out?',
 'Lmfao GOAT intro',
 'Hi neetcode . I want to know is python is good language to do DSA .does interviewer will allowed me to code in python .',
 'This is definitely a steal deal 🔥',
 'easiest purchase I’ve ever made',
 'Please do with swift languages for iOS dev like me 👨🏽\u200d💻 I appreciate all your effort and learning experience and hard work. Thanks 🙏',
 'I just purchased your course! When can we expect t

In [22]:
# Convert count columns to numeric columns
numeric_cols = ['subscribers', 'views', 'totalVideos']
channel_stats[numeric_cols] = channel_stats[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [23]:
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,NeetCode,215000,14981460,328,UU_mYaQAE6-71rjSN6CeCA-g


In [24]:
channel_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   channelName  1 non-null      object
 1   subscribers  1 non-null      int64 
 2   views        1 non-null      int64 
 3   totalVideos  1 non-null      int64 
 4   playlistId   1 non-null      object
dtypes: int64(3), object(2)
memory usage: 168.0+ bytes
